In [1]:
import pandas as pd
import numpy as np

In [2]:
# Create an embeeding 
df_items_master = pd.read_csv("Item_master.csv")
print("The number of rows and columns : ", df_items_master.shape)

The number of rows and columns :  (5931, 56)


In [3]:
df_items_master.columns

Index(['item', 'description', 'overview', 'matlType', 'MaterialStatus',
       'Stocked', 'organizationCode', 'Grade', 'specCode', 'Shape', 'ppapItem',
       'itemType', 'diameterMaximum', 'diameterMininum', 'ThicknessMaximum',
       'WidthMinimum', 'crossCornerMaximum', 'crossCornerMinimum',
       'widthMaximum', 'widthMinimum', 'cornerRadius', 'lengthMaximum',
       'lengthMinimum', 'quenchTemper', 'turned', 'sawCut1End', 'sawcutBoth',
       'stressRelieved', 'lpAnnealed', 'ground', 'shearOne', 'shearBoth',
       'spheriodizeAnneal', 'polished', 'chamfered1End', 'chamferedBoth',
       'coldDrawn', 'faceOneEnd', 'faceBoth', 'deburr', 'demagnetize',
       'oneHeatOnly', 'markHeatOne', 'markHeatBoth', 'annealed', 'pickled',
       'normalized', 'straighten', 'rollMark', 'rollMarkNote', 'roto',
       'magnetic', 'ultrasonicTest', 'eddyCurrent', 'specDesc', 'grainType'],
      dtype='object')

In [ ]:
# List of columns to keep
columns_to_keep = [
    'Grade',  'Shape', 'diameterMaximum', 'diameterMininum', 'ThicknessMaximum'
    ]

# Create new DataFrame with only selected columns
df_selected = df_items_master[columns_to_keep]

In [6]:
df_selected['row_values'] = df_selected.apply(lambda row: row.tolist(), axis=1)


C:\Users\dell\AppData\Local\Temp\ipykernel_22372\3527924951.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['row_values'] = df_selected.apply(lambda row: row.tolist(), axis=1)


In [9]:
df_selected['row_values']

0                     [1010, RD, 1.5, 1.498, 0.0]
1                    [1010, RD, 1.15, 1.148, 0.0]
2                   [1018, RD, 1.498, 1.496, 0.0]
3                   [1018, RD, 1.139, 1.137, 0.0]
4                 [1018, SQ, 0.8125, 0.8125, 0.0]
                          ...                    
5926    [Stressproof(R), RD, 0.5156, 0.5136, 0.0]
5927        [Stressproof(R), RD, 2.0, 1.995, 0.0]
5928      [Stressproof(R), RD, 0.875, 0.872, 0.0]
5929       [Stressproof(R), RD, 2.125, 2.12, 0.0]
5930           [Type 630, RD, 0.3145, 0.311, 0.0]
Name: row_values, Length: 5931, dtype: object

In [10]:
df_selected['length_row_values'] = df_selected['row_values'].apply(len)

C:\Users\dell\AppData\Local\Temp\ipykernel_22372\670563286.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['length_row_values'] = df_selected['row_values'].apply(len)


In [11]:
# Create a new DataFrame from the 'row_values' column
df_list_row_values= pd.DataFrame(df_selected["row_values"])
df_list_row_values

,row_values
0,"[1010, RD, 1.5, 1.498, 0.0]"
1,"[1010, RD, 1.15, 1.148, 0.0]"
2,"[1018, RD, 1.498, 1.496, 0.0]"
3,"[1018, RD, 1.139, 1.137, 0.0]"
4,"[1018, SQ, 0.8125, 0.8125, 0.0]"
...,...
5926,"[Stressproof(R), RD, 0.5156, 0.5136, 0.0]"
5927,"[Stressproof(R), RD, 2.0, 1.995, 0.0]"
5928,"[Stressproof(R), RD, 0.875, 0.872, 0.0]"
5929,"[Stressproof(R), RD, 2.125, 2.12, 0.0]"


In [ ]:
# vectorize using FAISS

In [14]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document

# Convert each row to string
text_data =  df_selected["row_values"].apply(lambda row: ' '.join(str(item) for item in row)).tolist()
metadata_items = df_items_master["item"].tolist()


In [15]:
# Load HuggingFace Embedding Model

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


C:\Users\dell\AppData\Local\Temp\ipykernel_22372\3389071243.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [16]:
embedding_model

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [17]:
new_docs = [
    Document(page_content=text, metadata={"Item": item})
    for text, item in zip(text_data, metadata_items)
]

In [18]:
new_docs

[Document(metadata={'Item': 116381}, page_content='1010 RD 1.5 1.498 0.0'),
 Document(metadata={'Item': 120863}, page_content='1010 RD 1.15 1.148 0.0'),
 Document(metadata={'Item': 118944}, page_content='1018 RD 1.498 1.496 0.0'),
 Document(metadata={'Item': 108697}, page_content='1018 RD 1.139 1.137 0.0'),
 Document(metadata={'Item': 120388}, page_content='1018 SQ 0.8125 0.8125 0.0'),
 Document(metadata={'Item': 120427}, page_content='1018 SQ 0.75 0.747 0.0'),
 Document(metadata={'Item': 124371}, page_content='1018 FL 0.0 0.0 0.5313'),
 Document(metadata={'Item': 125210}, page_content='1018 FL 0.0 0.0 0.9063'),
 Document(metadata={'Item': 127652}, page_content='1018 FL 0.0 0.0 0.5313'),
 Document(metadata={'Item': 127687}, page_content='1018 FL 0.0 0.0 0.7813'),
 Document(metadata={'Item': 127688}, page_content='1018 FL 0.0 0.0 0.75'),
 Document(metadata={'Item': 127689}, page_content='1018 FL 0.0 0.0 0.9063'),
 Document(metadata={'Item': 127690}, page_content='1018 FL 0.0 0.0 0.875')

In [19]:
vectorstore_new = FAISS.from_documents(new_docs, embedding_model)


In [20]:
vectorstore_new

In [21]:
def generate_values_string(filled_values: dict) -> str:
    
    keys = [ 'Grade',  'Shape', 'diameterMaximum', 'diameterMininum', 'ThicknessMaximum']

    # Step 1: Create a base dict with empty values
    empty_dict = {key: '' for key in keys}

    # Step 2: Merge the filled values
    final_dict = {**empty_dict, **filled_values}

    # Step 3: Convert all values to string and join them
    all_values_str = ', '.join(str(value) for value in final_dict.values())

    return all_values_str

In [22]:
# # item = 113422

filled_values = {
    'Grade': '4142SE',
    'Shape' : 'RD',
    'diameterMaximum' : 1.625,
    'diameterMininum' : 1.625,
    'ThicknessMaximum' : 0,
}


result_str = generate_values_string(filled_values)
print('result_str ----> ', result_str)
print("\n")

query = result_str
results = vectorstore_new.similarity_search_with_score(query, k=20)
for doc, score in results:
    print(f"Item: {doc.metadata['Item']}")
    print(f"Score: {score}")
    print(f"Text: {doc.page_content}")
    print("---")


result_str ---->  4142SE, RD, 1.625, 1.625, 0


Item: 128545
Score: 0.10878828167915344
Text: 4142SE RD 1.625 1.625 0.0
---
Item: 128542
Score: 0.12118285149335861
Text: 4142SE RD 1.625 1.619 0.0
---
Item: 128546
Score: 0.16291728615760803
Text: 4142SE RD 1.5 1.495 0.0
---
Item: 128439
Score: 0.2105354368686676
Text: 4142SE RD 1.25 1.25 0.0
---
Item: 110088
Score: 0.2282029390335083
Text: 4142SE RD 0.5625 0.5595 0.0
---
Item: 110942
Score: 0.22974923253059387
Text: 4142SE RD 0.5 0.496 0.0
---
Item: 110089
Score: 0.23214031755924225
Text: 4142SE RD 0.625 0.622 0.0
---
Item: 110994
Score: 0.2378629744052887
Text: 4142SE RD 0.515 0.51 0.0
---
Item: 128541
Score: 0.24183958768844604
Text: 4142SE RD 1.75 1.75 0.0
---
Item: 109312
Score: 0.24199526011943817
Text: 4142SE RD 0.4375 0.4345 0.0
---
Item: 128660
Score: 0.2443319857120514
Text: 4140 RD 1.625 1.625 0.0
---
Item: 110090
Score: 0.24953897297382355
Text: 4142SE RD 0.6875 0.6845 0.0
---
Item: 126137
Score: 0.2541990876197815
Text: 4142

In [23]:

filled_values = {
    'Grade': 1215,
    'Shape' : 'RD',
    'diameterMaximum' : '',
    'diameterMininum' : '',
    'ThicknessMaximum' : '',
}


result_str = generate_values_string(filled_values)
print('result_str ----> ', result_str)
print("\n")

query = result_str
results = vectorstore_new.similarity_search_with_score(query, k=20)
for doc, score in results:
    print(f"Item: {doc.metadata['Item']}")
    print(f"Score: {score}")
    print(f"Text: {doc.page_content}")
    print("---")


result_str ---->  1215, RD, , , 


Item: 109209
Score: 0.3568345308303833
Text: 1215 RD 1.3386 1.3366 0.0
---
Item: 109203
Score: 0.437034010887146
Text: 1215 RD 1.247 1.245 0.0
---
Item: 109266
Score: 0.4412316679954529
Text: 1215 RD 1.2813 1.2793 0.0
---
Item: 123037
Score: 0.44980281591415405
Text: 1215 RD 1.4375 1.4355 0.0
---
Item: 123842
Score: 0.44980281591415405
Text: 1215 RD 1.4375 1.4355 0.0
---
Item: 109212
Score: 0.44980281591415405
Text: 1215 RD 1.4375 1.4355 0.0
---
Item: 127858
Score: 0.44980281591415405
Text: 1215 RD 1.4375 1.4355 0.0
---
Item: 127624
Score: 0.4498479962348938
Text: 1215 RD 1.125 1.123 0.0
---
Item: 129838
Score: 0.4498479962348938
Text: 1215 RD 1.125 1.123 0.0
---
Item: 129451
Score: 0.4498479962348938
Text: 1215 RD 1.125 1.123 0.0
---
Item: 119276
Score: 0.4498479962348938
Text: 1215 RD 1.125 1.123 0.0
---
Item: 109190
Score: 0.4498479962348938
Text: 1215 RD 1.125 1.123 0.0
---
Item: 109191
Score: 0.4498479962348938
Text: 1215 RD 1.125 1.123 0.0
---
I

In [ ]:

filled_values = {
    'Grade': '',
    'Shape' : 'HEX',
    'diameterMaximum' : 0.6875,
    'diameterMininum' : 0.6855,
    'ThicknessMaximum' : 0,

}


result_str = generate_values_string(filled_values)
print('result_str ----> ', result_str)
print("\n")

query = result_str
results = vectorstore_new.similarity_search_with_score(query, k=20)
for doc, score in results:
    print(f"Item: {doc.metadata['Item']}")
    print(f"Score: {score}")
    print(f"Text: {doc.page_content}")
    print("---")


result_str ---->  , HEX, 0.6875, 0.6855, 0


Item: 108699
Score: 0.17414382100105286
Text: 1018 HEX 0.6875 0.6855 0.0
---
Item: 116955
Score: 0.17414382100105286
Text: 1018 HEX 0.6875 0.6855 0.0
---
Item: 116973
Score: 0.17414382100105286
Text: 1018 HEX 0.6875 0.6855 0.0
---
Item: 113053
Score: 0.17414382100105286
Text: 1018 HEX 0.6875 0.6855 0.0
---
Item: 127857
Score: 0.17414382100105286
Text: 1018 HEX 0.6875 0.6855 0.0
---
Item: 109937
Score: 0.17414382100105286
Text: 1018 HEX 0.6875 0.6855 0.0
---
Item: 120335
Score: 0.21927708387374878
Text: 1018 HEX 0.4375 0.4355 0.0
---
Item: 127310
Score: 0.22199980914592743
Text: 1018 HEX 1.437 1.437 0.0
---
Item: 108921
Score: 0.22910970449447632
Text: 1137 HEX 0.6875 0.6845 0.0
---
Item: 125571
Score: 0.23612335324287415
Text: 1018 HEX 0.75 0.748 0.0
---
Item: 109936
Score: 0.23863449692726135
Text: 1018 HEX 0.625 0.623 0.0
---
Item: 112946
Score: 0.24069693684577942
Text: 1018 HEX 0.3457 0.3437 0.0
---
Item: 108698
Score: 0.2428966164588928